<a href="https://colab.research.google.com/github/misharigot/kobe/blob/master/src/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the neural network to predict kobe's shots.

In [19]:
import sys; sys.path.insert(0, '..')  # Needed to make the import below work

from multiple_train_test_splits import MultipleTrainTestSplits

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing


import numpy as np
import csv
import pandas as pd
from scipy import stats

from keras.models import Sequential
from keras.layers import Dense
import keras
from sklearn import preprocessing
import tensorflow as ft

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [39]:
df = pd.read_csv('../../data/data.csv')

In [41]:
print(type(min(df['game_date'])))

<class 'str'>


In [ ]:
def get_months_elapsed(game_date: pd.Series):
    game_date
    
get_months_elapsed(df['game_date'])

In [34]:
def one_hot_encode(df: pd.DataFrame, encoder: preprocessing.OneHotEncoder = None) -> pd.DataFrame:
    """One-hot encode all categorical columns.
    Optionally provide an encoder. Use the training set encoder to one-hot encode the test set.
    """
     # Categorize all columns based on their data type
    categorical_columns = [
        'action_type',
        'combined_shot_type',
        'game_id',
        'season',
        'shot_type',
        'shot_zone_area',
        'shot_zone_basic',
        'shot_zone_range',
        'team_id',
        'team_name',
        'matchup',
        'opponent'
    ]

    temporal_columns = [
        'months_elapsed',
        'day_of_week'
    ]

    remaining_columns = [
        'period',
        'shot_distance',
        'shot_made_flag',  # y label
    ]

    excluded_columns = [
        'lat',
        'lon',
        'game_event_id', # Game event ids that are registered by the NBA probably, not relevant
        'shot_id',       # Just an auto-increment id, does not mean anything
    ]
    
    excluded_but_feature_engineered_columns = [
        'minutes_remaining',
        'seconds_remaining',
        'loc_x', # pythagoras
        'loc_y', # pythagoras
        'game_date', # months_elapsed_from_start & day_of_week
    ]

    # Convert relevant columns to categorical columns
    df[categorical_columns] = df[categorical_columns].astype('category')
    df_with_only_categoricals = df[categorical_columns]

    # One hot encode categorical columns
    if encoder is None:
        encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
        encoder.fit(df_with_only_categoricals)
    one_hot_encoded_df = pd.DataFrame(encoder.transform(df_with_only_categoricals).toarray())
    

    # Combine the one hot encoded part of the df with the remaining df
    non_categorical_df = df[remaining_columns]
    resulting_df = pd.concat([one_hot_encoded_df, non_categorical_df], axis=1)
    return resulting_df, encoder

In [35]:
def get_x(data: pd.DataFrame) -> pd.DataFrame:
    """Returns the features.
    """
    X = data.drop(columns=['shot_made_flag'])
    return X

def get_y(data: pd.DataFrame) -> pd.Series:
    """Returns the target.
    """
    Y = data['shot_made_flag'].copy()
    return Y


In [36]:
def preprocess(data: pd.DataFrame, encoder:preprocessing.OneHotEncoder = None) -> np.array:
    """Preprocess the raw kobe data from Kaggle.
    Optionally provide an encoder. Use the training set encoder to one-hot encode the test set.
    """
    df, encoder = one_hot_encode(data, encoder)
    
    return df, encoder

In [37]:


mtts = MultipleTrainTestSplits(csv_path='../../data/data.csv')

test_set = mtts.test_set

loss_and_metrics = []

for train_set, validation_set in mtts.train_validation_split(as_dataframe=True):
    # Preprocess the training set
    preprocessed_train_set, one_hot_encoder = preprocess(train_set)
    # Split the features from the target
    x_train = get_x(preprocessed_train_set)
    y_train = get_y(preprocessed_train_set)

    # Preprocess the validation set (use the one hot encoder that was fit on the training set)
    preprocessed_validation_set, _ = preprocess(validation_set, encoder=one_hot_encoder)
    # Split the features from the target
    x_validation = get_x(preprocessed_validation_set)
    y_validation = get_y(preprocessed_validation_set)

    scores = []
    
    knn = KNeighborsClassifier(3) # We set the number of neighbors to 15
    knn.fit(x_train, y_train.astype(int))
    y_predicted = knn.predict(x_validation)
    scores.append(accuracy_score(y_validation.astype(int), y_predicted.astype(int)))
    print('Loop:', 3 ,'accuracy ', accuracy_score(y_validation.astype(int), y_predicted.astype(int)))

    
# Results with old pre-processing
# Loop: 3 accuracy  0.5660634364662386
# Loop: 3 accuracy  0.5598365440747227
# Loop: 3 accuracy  0.5471881689044561

Loop: 3 accuracy  0.5680093403385873
Loop: 3 accuracy  0.5508853862619186
Loop: 3 accuracy  0.535318155283129


In [ ]:
def print_average_metrics(loss_and_metrics):
    # Get average accuracy
    accuracies = []
    for row in loss_and_metrics:
        accuracies.append(row[1])
    avg_accuracy = sum(accuracies) / len(accuracies)

    print('Average accuracy:', round(avg_accuracy, 4))


print_average_metrics(loss_and_metrics)

In [ ]:
classes = model.predict(x_validation, batch_size=128)
classes